In [49]:
import pandas as pd
from collections import Counter

import kagglehub
import shutil
import os

import shutil
import math
import random

import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt

Veriler ile ilgili bilgi edindiğimiz kısım

In [12]:
# CSV dosyasını oku
csv_path = "Data_Entry_2017.csv"
df = pd.read_csv(csv_path)


# Sadece tek bir etiketi olanları filtrele
single_label_df = df[df["Finding Labels"].str.contains(r"\|", na=False) == False]

# Hastalık etiketlerini say
label_counts = Counter(single_label_df["Finding Labels"].dropna())  # Boş olmayanları al



# # Birden çok hastalık etiketlerini de say 
# label_counts = Counter()
# for labels in df["Finding Labels"].dropna():  # Boş olmayanları al
#     label_counts.update(labels.split("|"))  # "|" ile ayrılanları ayır ve say

# Etiketleri çoktan aza doğru sırala
sorted_labels = label_counts.most_common()

# Sonuçları yazdır
print(f"Toplam {len(label_counts)} farklı hastalık etiketi var.")
print("Hastalık etiketleri (çoktan aza doğru):")
for label, count in sorted_labels:
    print(f"{label}: {count}")

Toplam 15 farklı hastalık etiketi var.
Hastalık etiketleri (çoktan aza doğru):
No Finding: 60361
Infiltration: 9547
Atelectasis: 4215
Effusion: 3955
Nodule: 2705
Pneumothorax: 2194
Mass: 2139
Consolidation: 1310
Pleural_Thickening: 1126
Cardiomegaly: 1093
Emphysema: 892
Fibrosis: 727
Edema: 628
Pneumonia: 322
Hernia: 110


Verileri Türlerine göre ayırdığımız kısım

In [14]:
def organize_images(df, source_folder = "archive/", destination_folder= "dataset/", chose = 1):
    """
    Görüntüleri etiketlere göre organize eden bir fonksiyon.

    Bu fonksiyon, verilen bir DataFrame'deki görüntü dosyalarını etiketlerine göre
    belirtilen hedef klasöre taşır veya kopyalar. Ayrıca, tüm görüntüleri tek bir
    klasöre toplama seçeneği sunar.

    Args:
        df (pd.DataFrame): Görüntü dosyalarının bilgilerini içeren DataFrame.
                           "Image Index" sütunu dosya adlarını, "Finding Labels" sütunu ise etiketleri içermelidir.
        source_folder (str): Kaynak klasörün yolu. Varsayılan olarak "archive/".
        destination_folder (str): Hedef klasörün yolu. Varsayılan olarak "dataset/".
        chose (int): İşlem türünü belirler:
                     - 1: Görüntüleri etiketlerine göre ayrı klasörlere taşır/kopyalar.
                     - 0: Tüm görüntüleri tek bir klasöre taşır/kopyalar.

    Returns:
        None

    Örnek:
        Tek etiketli görüntüleri organize etmek için:
        >>> organize_images(single_label_df, source_folder="archive/", destination_folder="dataset/", chose=1)
    """

    if chose == 0:
        # Genel hedef klasör (eğer chose = 0 ise tüm görüntüler buraya gider)
        general_folder = os.path.join(destination_folder, "single_label_images")

    for folder in os.listdir(source_folder):
        images_path = os.path.join(source_folder, folder, "images")

        if os.path.exists(images_path):  # Eğer images klasörü varsa

            for filename in os.listdir(images_path):  # İçindeki tüm görüntüleri tarayalım
                file_path = os.path.join(images_path, filename)  # Dosyanın tam yolu
                

                # CSV'den uygun etiket bulunuyor
                row = df.loc[df["Image Index"] == filename]  
                if not row.empty:
                    
                    if chose == 1:
                        label = row["Finding Labels"].values[0]  # Hastalık etiketi

                        # Etikete göre hedef klasör oluştur
                        label_folder = os.path.join(destination_folder, label)
                        os.makedirs(label_folder, exist_ok=True)

                        # Dosyayı taşı
                        destination_path = os.path.join(label_folder, filename)
                        shutil.copy(file_path, destination_path)


                    elif chose == 0:
                        os.makedirs(general_folder, exist_ok=True)

                        # Tüm görüntüleri tek klasöre kopyala
                        destination_path = os.path.join(general_folder, filename)
                        shutil.copy(file_path, destination_path)


    print("Tüm görüntüler etiketlere göre başarıyla taşındı!")

In [ ]:
# Tek etiketli görüntüleri organize et
organize_images(single_label_df)

Tüm görüntüler etiketlere göre başarıyla taşındı!


In [16]:
# Tek etiketli görüntüleri tek bir klasöre kopyala
organize_images(single_label_df, destination_folder= "single_dataset/", chose= 0)

Tüm görüntüler etiketlere göre başarıyla taşındı!


In [46]:
def egitim_dosya_hazirla(source_folder= 'dataset/',
                         destination_folder= "data_model/",
                         excluded_folder = 'No Finding',
                         num_images_per_class = 1000,
                         train_split_ratio = 0.85,
                         test_split_ratio = 0.10
                         ):
    

    os.makedirs(destination_folder, exist_ok=True)
    disease_folders = [d for d in os.listdir(source_folder)]



    for disease_name in disease_folders:
        # Hariç tutulan klasörü atla
        if disease_name == excluded_folder:
            continue

        disease_path = os.path.join(source_folder, disease_name)

        all_images = [f for f in os.listdir(disease_path)]

        num_available_images = len(all_images)
        if num_available_images < num_images_per_class:
            print(f" Uyarı: '{disease_name}' klasöründe yeterli resim yok ({num_available_images} < {num_images_per_class}). Bu sınıf atlanıyor.")
            continue

        # Rastgele 1000 resim seç
        selected_images = random.sample(all_images, num_images_per_class)  

        # Seçilen resimleri karıştır
        random.shuffle(selected_images)

        # Bölme için resim sayılarını hesapla
        num_train = int(num_images_per_class * train_split_ratio)
        num_test = int(num_images_per_class * test_split_ratio)
        # Küsüratları doğrulama setine ekleyerek tam sayıya tamamla
        num_val = num_images_per_class - num_train - num_test

        # Resimleri bölümlere ayır
        train_images = selected_images[:num_train]
        test_images = selected_images[num_train:num_train + num_test]
        val_images = selected_images[num_train + num_test:]
    
        # Hedef klasörleri oluştur
        split_mapping = {
        'train': train_images,  # Eğitim seti
        'validation': val_images, # Doğrulama seti
        'test': test_images     # Test seti
        }

        for split_name, image_list in split_mapping.items():

            dest_dir = os.path.join(destination_folder, split_name, disease_name)
            os.makedirs(dest_dir, exist_ok=True) # İç içe klasörleri oluşturur

            # Resimleri kopyalama işlemi
            for image_filename in image_list:
                source_file_path = os.path.join(disease_path, image_filename)
                dest_file_path = os.path.join(dest_dir, image_filename)
                
                shutil.copy(source_file_path, dest_file_path) # Sadece dosyayı kopyalar
                

In [47]:
egitim_dosya_hazirla()

 Uyarı: 'Edema' klasöründe yeterli resim yok (628 < 1000). Bu sınıf atlanıyor.
 Uyarı: 'Emphysema' klasöründe yeterli resim yok (892 < 1000). Bu sınıf atlanıyor.
 Uyarı: 'Fibrosis' klasöründe yeterli resim yok (727 < 1000). Bu sınıf atlanıyor.
 Uyarı: 'Hernia' klasöründe yeterli resim yok (110 < 1000). Bu sınıf atlanıyor.
 Uyarı: 'Pneumonia' klasöründe yeterli resim yok (322 < 1000). Bu sınıf atlanıyor.


In [59]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # grayscale için
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [60]:
data_dir = 'data_model/'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
    for x in ['train', 'validation', 'test']
}

class_names = image_datasets['train'].classes
print("Sınıflar:", class_names)


Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']


In [61]:
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
    for x in ['train', 'validation', 'test']
}

In [62]:
model = models.resnet18(pretrained=True)

# İlk katman RGB (3 kanal) bekliyor. 1 kanala uyarladık.
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Son katmanı sınıf sayısına göre değiştirdik.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [65]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    return model


In [ ]:
trained_model = train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10)


Epoch 1/10
--------------------
